In [14]:
import pandas as pd
import numpy as np
import scipy

from statsmodels.sandbox.stats.multicomp import multipletests 

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо. 

In [10]:
aucs = pd.read_csv('AUCs.txt', delimiter='\t', index_col=0)

In [11]:
aucs.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
adult (sample),0.763,0.768,0.771,0.798
breast cancer,0.599,0.591,0.590,0.569
breast cancer wisconsin,0.954,0.971,0.968,0.967
cmc,0.628,0.661,0.654,0.657
ionosphere,0.882,0.888,0.886,0.898


In [19]:
wilcoxon_data = []

for i, lhs_column in enumerate(aucs.columns):
    for j, rhs_column in enumerate(aucs.columns):
        if i >= j:
            continue
        
        wilcoxon, p = scipy.stats.wilcoxon(aucs[lhs_column], aucs[rhs_column])
        wilcoxon_data.append([lhs_column, rhs_column, wilcoxon, p])

In [25]:
df = pd.DataFrame.from_records(wilcoxon_data)
df.columns = ['first', 'second', 'wilcoxon_stat', 'p']

In [26]:
df.sort_values(by='p', ascending=True)

,first,second,wilcoxon_stat,p
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015906
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [28]:
len(df[df.p < .05])

4

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [30]:
reject, p_corrected, a1, a2 = multipletests(df.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

In [32]:
sum(reject)

0

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга? 

In [33]:
reject, p_corrected, a1, a2 = multipletests(df.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [34]:
sum(reject)

3